In [138]:
import math
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import numpy as np
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from boruta import BorutaPy
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet

In [2]:
df = pd.read_csv("..\\dataset_machine_learning.csv")

In [3]:
df

,date,price,bedrooms,bathrooms,sqm_living,sqm_lot,floors,waterfront,view,condition,grade,yr_built,renovated,yr_renovated,zipcode,city_encoded,lat,long,sqm_living15,sqm_lot15
0,0,790000,3,2.50,241.55,441.29,1.0,False,0,4,9,1951,False,1951,98117,11,47.6833,-122.400,221.11,441.29
1,0,555000,4,2.50,307.51,603.87,2.0,False,0,3,8,2012,False,2012,98065,15,47.5150,-121.870,221.11,464.52
2,0,440000,4,2.25,200.67,754.28,1.0,False,0,3,8,1966,False,1966,98006,21,47.5443,-122.177,171.87,836.13
3,0,295000,2,2.50,151.43,127.09,2.0,False,0,3,7,2009,False,2009,98106,11,47.5489,-122.363,147.72,214.23
4,0,470000,5,2.50,205.32,896.98,1.0,False,0,3,8,1976,False,1976,98011,14,47.7698,-122.222,193.24,802.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19251,377,183000,3,1.00,108.70,680.05,1.0,False,0,3,7,1962,False,1962,98188,11,47.4685,-122.282,189.52,680.05
19252,377,359000,2,2.75,127.28,105.91,2.0,False,0,3,8,2009,False,2009,98133,11,47.7052,-122.343,127.28,101.26
19253,378,450000,5,2.00,173.73,682.28,1.5,False,0,3,7,1960,False,1960,98007,21,47.5951,-122.144,173.73,710.71
19254,387,445500,2,1.75,129.14,619.66,1.0,False,0,3,6,1941,False,1941,98115,11,47.6914,-122.308,85.47,592.72


Mise en forme des dataset Train/Test

In [4]:
X1 = df['sqm_living'].values.reshape(-1,1)
X2 = df['grade'].values.reshape(-1,1)
X3 = df[['grade', 'sqm_living']]
X4 = df.drop(columns=['price'])
y = df['price'].values.reshape(-1,1)


X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size=0.2)
X2_train, X2_test, y2_train, y2_test = train_test_split(X1, y, test_size=0.2)
X3_train, X3_test, y3_train, y3_test = train_test_split(X1, y, test_size=0.2)
X4_train, X4_test, y4_train, y4_test = train_test_split(X1, y, test_size=0.2)

df_mean_price = df.groupby('grade').agg(mean_price=('price', 'mean'))

Xmean = df_mean_price.index.values.reshape(-1,1)
ymean = df_mean_price['mean_price'].values.reshape(-1,1)

Création des modeles

In [139]:
eNet_model = ElasticNet()

poly_model = PolynomialFeatures(6)
linear_model_for_poly = LinearRegression()

RandomForest_model = RandomForestRegressor(max_depth=2)

model_SVR = SVR( kernel = 'linear' , gamma = 'auto')

knn_model = KNeighborsRegressor(n_neighbors=3)

Modele lineaire ElasticNet

In [140]:
linear_param_grid = {"max_iter": [1, 5, 10, 50, 100, 500, 1000, 5000, 10000],
                      "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                      "l1_ratio": np.arange(0.0, 1.0, 0.1), 
                      'tol' : [0.00001, 0.0001, 0.001]}


grid = GridSearchCV(eNet_model, linear_param_grid, scoring='r2', cv=10)
grid.fit(X4_train, y4_train)

print(f"Best score = {grid.best_score_}")
print(f"Best param = {grid.best_params_}")

eNet_model = grid.best_estimator_

eNet_model.score(X4_test, y4_test)

c:\Users\Consultant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.931e+14, tolerance: 1.474e+11 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
c:\Users\Consultant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.856e+14, tolerance: 1.444e+11 Linear regression models with null weight for the l1 regularization term are more ef

Best score = 0.4661038652238762
Best param = {'alpha': 10, 'l1_ratio': 0.0, 'max_iter': 1}


c:\Users\Consultant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.324e+14, tolerance: 1.619e+11 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


0.4821534890791733

In [ ]:
df = pd.DataFrame(grid.cv_results_)
df

Modele KNN

In [149]:
knn_param_grid = {"n_neighbors" : range(1,50), 
                  "metric" : ["euclidean", "manhattan"]}

grid = GridSearchCV(knn_model, knn_param_grid, scoring='r2', cv =10)
grid.fit(X4_train, y4_train)

print(f"Best score = {grid.best_score_}")
print(f"Best param = {grid.best_params_}")

knn_model = grid.best_estimator_

knn_model.score(X4_test, y4_test)

Best score = 0.48920134702750495
Best param = {'metric': 'euclidean', 'n_neighbors': 49}


0.492818927996028